<a href="https://colab.research.google.com/github/jmohsbeck1/jpmc_mle/blob/week-Apr.-17/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://raw.githubusercontent.com/fenago/datasets/main/data.csv

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/main/data.csv")
df.sample(3)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
2306,2014-06-11 00:00:00,852600.0,4.0,2.5,3320,11901,2.0,0,0,5,2650,670,1977,0,5334-5394 134th Ave SE,Bellevue,WA 98006,USA
3145,2014-06-23 00:00:00,560000.0,4.0,2.5,2300,7989,2.0,0,0,3,2300,0,1990,2009,22912 NE 19th Dr,Sammamish,WA 98074,USA
2070,2014-06-06 00:00:00,585000.0,4.0,2.5,2430,4747,2.0,0,0,3,2430,0,2008,0,360 SE Bush St,Issaquah,WA 98027,USA


In [ ]:
to_drop = ['date', 'street','statezip','country']
df = df.drop(to_drop, axis = 1)
df.head(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,Shoreline
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,Seattle
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,Kent
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,Bellevue
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,Redmond


In [6]:
# How old is the house?
df['house_age'] = [2023 - yr_built for yr_built in df['yr_built']]

# Was the house renovated and was the renovation recent?
df['was_renovated'] = [1 if yr_renovated != 0 else 0 
    for yr_renovated in df['yr_renovated']]
df['was_renovated_10_yrs'] = [1 if (2023 - yr_renovated) <= 10 
    else 0 for yr_renovated in df['yr_renovated']]
df['was_renovated_30_yrs'] = [1 if 10 < (2023 - yr_renovated) <= 30
    else 0 for yr_renovated in df['yr_renovated']]

# Drop original columns
df = df.drop(['yr_built', 'yr_renovated'], axis=1)
df.head()

KeyError: ignored